In [38]:
import sys
import os

abspath = r'/home/dev/git/raptor-stats/lambdas'
sys.path.append(os.path.abspath(abspath))

os.environ['ENV'] = 'dev'

from lambdas.Common.cast_frame import add_computed_cols, cast_frame
from lambdas.Common.common import get_df

from lambdas.PveSkill.pve_skill import replay_details_file_name
from lambdas.Common.gamesettings import nuttyb_hp_multiplier

import polars as pl

nuttyb_hp_enum = pl.Enum(
    ['Epic', 'Epic+', 'Epic++', 'Epicer+', 'Epicer++', 'Epicest'],
)

nuttyb_hp_df = pl.DataFrame(
    [(hp, tweak) for hp, tweaks in nuttyb_hp_multiplier.items() for tweak in tweaks],
    schema=['nuttyb_hp', 'tweakdefs1'],
)

asdf = (
    pl.read_parquet(replay_details_file_name)
    .join(
        nuttyb_hp_df,
        on='tweakdefs1',
        how='left',
    )
    .cast(
        {
            'nuttyb_hp': nuttyb_hp_enum,
        },
        strict=True,
    )
)

In [46]:
print(asdf[['startTime']].min())
print(asdf[['startTime']].max())

shape: (1, 1)
┌─────────────────────────┐
│ startTime               │
│ ---                     │
│ datetime[ns, UTC]       │
╞═════════════════════════╡
│ 2024-03-10 22:53:40 UTC │
└─────────────────────────┘
shape: (1, 1)
┌─────────────────────────┐
│ startTime               │
│ ---                     │
│ datetime[ns, UTC]       │
╞═════════════════════════╡
│ 2024-06-07 11:20:03 UTC │
└─────────────────────────┘


In [59]:
import plotly.graph_objects as go

numbers = asdf.filter(pl.col('raptors').eq(True))['raptor_graceperiodmult'].to_list()
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=numbers,
        name='Markers and Text',
        texttemplate='%{x}',
        textfont_size=20,
        # textposition='inside',
        # xbins={'start': 0, 'end': 10, 'size': 0.2},
        # nbinsx=20,
    )
)

fig.show()

In [60]:
numbers = asdf.filter(pl.col('raptors').eq(True), pl.col('nuttyb_hp').is_not_null())[
    'raptor_queentimemult'
].to_list()
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=numbers,
        name='count',
        texttemplate='%{x}',
        textfont_size=20,
        # xbins={'start': 0, 'end': 10, 'size': 0.2},
        # nbinsx=20,
    )
)

fig.show()

In [61]:
import plotly.graph_objects as go

numbers = asdf.filter(pl.col('scavengers').eq(True))['scav_bosstimemult'].to_list()
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=numbers,
        name='count',
        texttemplate='%{x}',
        textfont_size=20,
        # xbins={'start': 0, 'end': 10, 'size': 0.2},
        # nbinsx=10,
    )
)

fig.show()

In [79]:
pl.Config(tbl_rows=50, tbl_cols=111, tbl_width_chars=4001)
asdf.filter(pl.col('raptors').eq(True) & pl.col('raptor_graceperiodmult').ge(5))[
    'raptor_endless'
].value_counts()

raptor_endless,count
u8,u32
1,64
0,192
